In [26]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

Выведем список категорий текстов:

In [27]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [39]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

vectorizer = CountVectorizer(lowercase=True, stop_words=ENGLISH_STOP_WORDS,
                             analyzer='word', binary=True)
vectorizer.fit(newsgroups_train.data)

CountVectorizer(analyzer='word', binary=True, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None,
        stop_words=frozenset({'per', 'around', 'so', 'mill', 'except', 'enough', 'she', 'nobody', 'whole', 'somewhere', 'inc', 'beside', 'twelve', 'which', 'almost', 'you', 'elsewhere', 'whereupon', 'its', 'former', 'everywhere', 'although', 'there', 'be', 'were', 'anyhow', 'bottom', 'at', 'him', 'myself', ...however', 'as', 'next', 'become', 'fill', 'they', 'once', 'without', 'them', 'therefore', 'anyone'}),
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

Переведем весь набор текстов обучающего датасета в набор векторов, получим матрицу ```X_train```:

In [40]:
X_train = vectorizer.fit_transform(newsgroups_train.data)


Задача: запустить модель LDA и Gibbs Sampling с числов тегов 20. Вывести топ-10 слов по каждому тегу. Соотнести полученные теги с тегами из датасета, сделать выводы. 

In [30]:
from tqdm import tqdm

In [31]:
tag = 20
nk=np.zeros(tag)
ndk=np.zeros(tag*X_train.shape[0]).reshape(X_train.shape[0],tag)
nkw=np.zeros(tag*X_train.shape[1]).reshape(tag,X_train.shape[1])
doc, word = X_train.nonzero()
z = np.random.choice(tag, len(doc))
for i, j, k in zip(doc, word, z):
    ndk[i, k] += 1
    nkw[k, j] += 1
    nk[k] += 1

In [36]:
def lda(ndk, nkw, nk, z, doc, word, tag, a1, a2, it):    
    for i in tqdm(range(it)):
        for j in range(len(doc)):
            cword = word[j]
            cdoc = doc[j]
            ctag = z[j]
            ndk[cdoc, ctag] -= 1
            nkw[ctag, cword] -= 1
            nk[ctag] -= 1
            p = (ndk[cdoc, :] + a1) * (nkw[:, cword] + a2[cword]) / (nk + a2.sum())
            z[j] = np.random.choice(np.arange(tag), p=p / p.sum())
            ndk[cdoc, z[j]] += 1
            nkw[z[j], cword] += 1
            nk[z[j]] += 1
    return z, nkw, ndk, nk

In [37]:
z,nkw,ndk,nk=lda(ndk, nkw, nk, z, doc, word, 20, 2*np.ones(20), 2*np.ones(X_train.shape[1]), 50)

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [24:55<00:00, 30.27s/it]


In [38]:
A = np.argsort(nkw, axis=1)[:, -10:]
for i in range(20):
    matrix = np.zeros((1, X_train.shape[1]))
    for j in A[i]:
        matrix[0, j] = 1
    print(i+1,'\t'.join(vectorizer.inverse_transform(matrix)[0]))

1 car	doing	getting	heard	line	list	lots	money	quite	reply
2 bible	christian	christians	claim	jesus	life	man	mean	reason	says
3 bad	big	car	cause	deal	doing	heard	problems	wrong	yes
4 card	computer	disk	drive	hi	memory	pc	sale	software	video
5 1993	april	earth	low	national	program	research	science	space	university
6 article	car	difference	large	left	list	remember	seen	trying	won
7 chip	clipper	encryption	government	key	keys	law	phone	private	public
8 couple	friend	game	getting	makes	simple	source	stuff	unless	wanted
9 11	14	17	20	30	game	games	play	season	team
10 ago	anybody	bad	change	goes	oh	remember	stuff	won	yes
11 banks	geb	gordon	intellect	isn	pitt	shameful	skepticism	soon	surrender
12 ago	couple	free	getting	heard	looked	pretty	remember	similar	thought
13 application	code	file	files	ftp	program	send	version	window	write
14 big	getting	job	kind	line	posted	quite	reading	reason	times
15 big	idea	level	looks	mind	note	problems	seen	sorry	unless
16 ago	anybody	big	guy	money	pay	prob

Попробуем соотнести полученные результаты со списком категорий текстов:      
 'alt.atheism' - 11       
 'comp.graphics' - 4,13,15      
 'comp.os.ms-windows.misc' - 4,13,15      
 'comp.sys.ibm.pc.hardware' - 4,13,15      
 'comp.sys.mac.hardware' - 4,13,15      
 'comp.windows.x' - 4,13,15     
 'misc.forsale' - 4     
 'rec.autos' - 1,3,6,17,18      
 'rec.motorcycles' - 1,3,6,17,18      
 'rec.sport.baseball' - 9      
 'rec.sport.hockey' - 9     
 'sci.crypt' - 7     
 'sci.electronics' - 4      
 'sci.med'      
 'sci.space' - 5      
 'soc.religion.christian' - 2      
 'talk.politics.guns' - 19      
 'talk.politics.mideast' - 19     
 'talk.politics.misc'       
 'talk.religion.misc'     
 К некоторым категория нельзя однозначно подобрать тэги, как и к некоторым тэгам нельзя однозначно подобрать категории. Также можно сказать о том, что к некоторым категориям невозможно подобрать тэги, потому что нет слов специализирующихся именно в той или другой категории. Но также можно заметить, что большая часть тэгов разобралась по категориям.